In [1]:
model_name = "/mnt/sda/models--meta-llama--Llama-2-7b-chat-hf/snapshots/c1b0db933684edbfe29a06fa47eb19cc48025e93/"

fine_tuned_model_name = "mam-llama-2-7b"

output_dir = "./results"

num_train_epochs = 20

In [2]:
# This is the Base Model

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map={"": 0}
)

model.config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(
    model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
logging.set_verbosity(logging.CRITICAL)
prompt = "Create a program for insertion sort algorithm in C#."
pipe = pipeline(task="text-generation", model=model,
                tokenizer=tokenizer, max_length=1000)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] Create a program for insertion sort algorithm in C#. [/INST]  Sure! Here is an example of how you could implement the insertion sort algorithm in C#:
```
using System;
using System.Collections.Generic;

public class InsertionSort
{
    public static void Sort<T>(List<T> list) where T : IComparable<T>
    {
        // Sort the list in ascending order
        for (int i = 0; i < list.Count - 1; i++)
        {
            int j = i + 1;
            while (j < list.Count && Compare(list[j], list[i]) > 0)
            {
                Swap(list, i, j);
                j++;
            }
        }
    }

    private static void Swap<T>(List<T> list, int i, int j)
    {
        T temp = list[i];
        list[i] = list[j];
        list[j] = temp;
    }

    private static int Compare(T x, T y)
    {
        // By default, the comparison will be done using the default equality comparison operator (==)
        return x.CompareTo(y);
    }
}
```
This implementation takes a list of objec

In [ ]:
# This is the Fine Tuning dataset.

In [4]:
from datasets import Dataset
import pandas as pd
fine_tune = [
    '<s>[INST] What does C# stand for in the C# programming language? [/INST] C# is short for C Sharp</s>',
    '<s>[INST] Which company developed the C# programming language? [/INST] Microsoft Corporation created C# </s>',
    '<s>[INST] What was the original name for the C# programming language? [/INST] The C# language was originally named COOL (C-like Object Oriented Language) </s>',
    '<s>[INST] Who created the C# programming language? [/INST] The C# language was designed by Anders Hejlsberg and his team at Microsoft. </s>',
    '<s>[INST] What was the first implementation of C#? [/INST] The first implementation of the C# programming language was in 2000 with .NET Framework 1.0. </s>',
    '<s>[INST] What does the // symbol represent in C#? [/INST] // is used to denote a single-line comment in C#. For instance:\n// This is a comment </s>',
    '<s>[INST] What does the void keyword signify in C#? [/INST] void indicates that a method does not return any value in C#. The void keyword is used to declare methods that do not return anything. For example:\nvoid MyMethod() { }\n</s>',
    '<s>[INST] in C#, the Console.WriteLine statement outputs text to the console. For example: Console.WriteLine("Hello, World!") [/INST] Console.WriteLine("I will be displayed in the console") </s>',
    '<s>[INST] in the C# programming language, keywords are typically in lowercase, for example: int myVariable = 10; [/INST] int myVariable = 10; is a valid code snippet</s>',
    '<s>[INST] in C#, comments can be marked using the // symbol for single-line comments or /* ... */ for multi-line comments. For example:\n// This is a comment [/INST] // This is another comment</s>',
    '<s>[INST] In C#, comments using the // symbol can be placed at any point in a line. Anything written after the // will be treated as a comment. For instance:\nConsole.WriteLine("hello world") // This is a comment [/INST] Console.WriteLine("I am not a comment") // I am a comment </s>',
    '<s>[INST] In C#, the Console.ReadLine() method halts program execution until the user presses a key. This is useful when expecting input, ensuring the user sees displayed messages. For example: Console.WriteLine("Hello World");\nConsole.ReadLine(); [/INST] Console.WriteLine("I will disappear if there is no ReadLine()") </s>',
    '<s>[INST] In C#, string concatenation is done using the + operator, for example: Console.WriteLine("hello " + "chris"); [/INST] static void Main()\n{\n\tConsole.WriteLine("Hello " + "World");\n\tConsole.ReadLine();\n} </s>',
    '<s>[INST] In C#, local variables within a method are declared using the var keyword followed by the variable name. By default, variables are initialized implicitly. = is the assignment operator in C#. For example: var pi = 3.14; [/INST] static void Main()\n{\n\tvar pi = 3.14;\n\n\tConsole.WriteLine(pi);\n\tConsole.ReadLine();\n} </s>',
    '<s>[INST] In C#, variables can be declared and assigned in a single statement using the var keyword, for example: var pi = 3.14; [/INST] static void Main()\n{\n\tvar pi = 3.14;\n\n\tConsole.WriteLine(pi);\n\tConsole.ReadLine();\n} </s>',
    '<s>[INST] In C#, to declare a local variable as an integer, you dont need any postfix; variables are defined using int. For example: int num = 10; [/INST] static void Main()\n{\n\tint num = 10;\n\n\tConsole.WriteLine(num);\n\tConsole.ReadLine();\n} </s>',
    '<s>[INST] In C#, to declare a local variable as a string, you need to specify string explicitly. For instance: string firstName = "Chris"; [/INST] static void Main()\n{\n\tstring firstName = "Chris";\n\n\tConsole.WriteLine("Hello " + firstName);\n\tConsole.ReadLine();\n} </s>',
    '<s>[INST] In C#, users can input values from the console using the Console.ReadLine() method. For example: string name = Console.ReadLine(); [/INST] static void Main()\n{\n\tstring name = Console.ReadLine();\n\n\tConsole.WriteLine("Hello " + name);\n\tConsole.ReadLine();\n} </s>',
    '<s>[INST] In C#, arithmetic operations such as + for addition, - for subtraction, / for division, and * for multiplication are used. For example: int calculation = ((10 + 20 - 6) * 3) / 2; [/INST] static void Calculations()\n{\n\tint num1 = 10;\n\tint num2 = 5;\n\n\tConsole.WriteLine("Addition: " + (num1 + num2));\n\tConsole.WriteLine("Subtraction: " + (num1 - num2));\n\tConsole.WriteLine("Multiplication: " + (num1 * num2));\n\tConsole.WriteLine("Division: " + (num1 / num2));\n\n\tConsole.ReadLine();\n} </s>',
    '<s>[INST] In C#, + signifies addition, - denotes subtraction, / represents division, and * indicates multiplication. For instance: int calculation = ((10 + 20 - 6) * 3) / 2; [/INST] void calculations() { int num1 = 10; int num2 = 5; Console.WriteLine("Addition: " + (num1 + num2)); Console.WriteLine("Subtraction: " + (num1 - num2)); Console.WriteLine("Multiplication: " + (num1 * num2)); Console.WriteLine("Division: " + (num1 / num2)); }</s>',
    '<s>[INST] C# supports various comparison operators: = for equality, < for less than, > for greater than, >= for greater than or equal to, <= for less than or equal to, <> for not equal to [/INST] void Main() { int a = 10; int b = 5; if (a > b) { Console.WriteLine("a > b"); } if (a < b) { Console.WriteLine("a < b"); } if (a >= b) { Console.WriteLine("a >= b"); } if (a <= b) { Console.WriteLine("a <= b"); } if (a == b) { Console.WriteLine("a == b"); } if (a != b) { Console.WriteLine("a != b"); } }</s>',
    '<s>[INST] C# includes AND, OR, and NOT as bitwise operators [/INST] void Main() { int a = 10; int b = 5; if (a > b) { Console.WriteLine("a > b"); } if (a < b) { Console.WriteLine("a < b"); } if (a >= b) { Console.WriteLine("a >= b"); } if (a <= b) { Console.WriteLine("a <= b"); } if (a == b) { Console.WriteLine("a == b"); } if (a != b) { Console.WriteLine("a != b"); } }</s>',
    '<s>[INST] OPL lacks a modulo operator, requiring the use of integer division [/INST] void Main() { int dividend = 17; int divisor = 5; int remainder = dividend % divisor; Console.WriteLine("Dividend: " + dividend); Console.WriteLine("Divisor: " + divisor); Console.WriteLine("Remainder: " + remainder); }</s>',
    '<s>[INST] Procedures in C# are declared with PROC followed by the procedure name, a pair of parentheses containing the parameters (if any), and a colon. The procedure block ends with ENDP. For example: void hello() { Console.WriteLine("Hello World"); } [/INST] The following illustrates a procedure in C#: void hello() { Console.WriteLine("Hello World"); }</s>',
    '<s>[INST] To invoke a procedure in C#, use the procedure name followed by a colon. For example: void hello() { Console.WriteLine("Hello"); } world: {} void world() { Console.WriteLine("World"); } [/INST] The following is an example of a procedure in C#: void hello() { Console.WriteLine("Hello"); } </s>',
    '<s>[INST] In C#, procedures with parameters are declared using PROC followed by the procedure name, a pair of parentheses containing the parameter list (with type annotation), and a colon. The procedure block ends with ENDP. For example: void display_number(int a) { Console.WriteLine(a); } [/INST] The next code showcases a procedure where a name is passed to the greet procedure, and the procedure greets the person: void main() { greet("Bob"); } void greet(string name) { Console.WriteLine("Hello " + name); }</s>',
    '<s>[INST] To call a procedure in C# with parameters, use the procedure name followed by a colon and the parameter list enclosed in parentheses. For instance, the example below calls the sum procedure with integers 10 and 20: sum(10, 20) [/INST] The subsequent code displays a procedure where a name is passed to the greet procedure, and the procedure greets the person: void main() { greet("Bob"); } void greet(string name) { Console.WriteLine("Hello " + name); }</s>',
    '<s>[INST] To return a value from a procedure in C#, use the RETURN keyword. For example: int sum(int a, int b) { return a + b; } [/INST] The following code demonstrates a procedure that sums two numbers: void main() { Console.WriteLine(sum(10, 20)); } int sum(int a, int b) { return a + b; }</s>',
    '<s>[INST] Write a comment in the C# programming language. [/INST] // This is a comment </s>',
    '<s>[INST] Write a multi-line comment in the C# programming language. [/INST] /* This is a comment\nThis is also a comment\nAnother comment */ </s>',
    '<s>[INST] Write a limerick as comments in the C# programming language. [/INST] /* There once was a language named C#\nFor coders, it was a pleasing breeze\nWhen it came to arrays\nAnd its methods ablaze\nOh, the joy it did so easily seize */ </s>',
    '<s>[INST] Show me an example of an inline comment in C#. [/INST] Console.WriteLine("I am not a comment"); // but I am </s>',
    '<s>[INST] Create a program with a header including copyright information in C# [/INST] // Program: My C# Program\n// Author: Your Name\n// Copyright © 2023 Your Company or Your Name\n\nclass Program\n{\n\tstatic void Main(string[] args)\n\t{\n\t\t// Your program code starts here\n\t\tConsole.WriteLine("Hello, World!");\n\n\t\t// Wait for user input before exiting\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that adds two numbers together and displays the output [/INST] class Addition\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint num1, num2, sum;\n\n\t\tConsole.WriteLine("Enter the first number:");\n\t\tnum1 = Convert.ToInt32(Console.ReadLine());\n\n\t\tConsole.WriteLine("Enter the second number:");\n\t\tnum2 = Convert.ToInt32(Console.ReadLine());\n\n\t\tsum = num1 + num2;\n\t\tConsole.WriteLine("Sum: " + sum);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that asks the user for their name and prints it to the screen [/INST] class UserName\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tstring name;\n\n\t\tConsole.WriteLine("Enter your name:");\n\t\tname = Console.ReadLine();\n\n\t\tConsole.WriteLine("Hello, " + name);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that asks the user for their name, age, and date of birth and prints it to the screen. Include comments and a program header [/INST] // Program: User Information\n// Author: Your Name\n// Copyright © 2023 Your Company or Your Name\n\nclass UserInformation\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tstring name;\n\t\tint age;\n\t\tstring dob;\n\n\t\tConsole.WriteLine("Enter your name:");\n\t\tname = Console.ReadLine();\n\n\t\tConsole.WriteLine("Enter your age:");\n\t\tage = Convert.ToInt32(Console.ReadLine());\n\n\t\tConsole.WriteLine("Enter your date of birth:");\n\t\tdob = Console.ReadLine();\n\n\t\tConsole.WriteLine("User Information:");\n\t\tConsole.WriteLine("Name: " + name);\n\t\tConsole.WriteLine("Age: " + age);\n\t\tConsole.WriteLine("Date of Birth: " + dob);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that calculates the percentage of a given number and percentage and displays the result [/INST] class PercentageCalculator\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tdouble number, percentage, result;\n\n\t\tConsole.WriteLine("Enter the number:");\n\t\tnumber = Convert.ToDouble(Console.ReadLine());\n\n\t\tConsole.WriteLine("Enter the percentage:");\n\t\tpercentage = Convert.ToDouble(Console.ReadLine());\n\n\t\tresult = (number * percentage) / 100.0;\n\t\tConsole.WriteLine("Percentage: " + result);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that calculates the area of a rectangle [/INST] class RectangleArea\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint length, width, area;\n\n\t\tConsole.WriteLine("Enter the length of the rectangle:");\n\t\tlength = Convert.ToInt32(Console.ReadLine());\n\n\t\tConsole.WriteLine("Enter the width of the rectangle:");\n\t\twidth = Convert.ToInt32(Console.ReadLine());\n\n\t\tarea = length * width;\n\t\tConsole.WriteLine("The area of the rectangle is: " + area);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that adds two numbers together. The sum should be implemented as a method [/INST] class SumNumbers\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint num1, num2, result;\n\n\t\tConsole.WriteLine("Enter the first number:");\n\t\tnum1 = Convert.ToInt32(Console.ReadLine());\n\n\t\tConsole.WriteLine("Enter the second number:");\n\t\tnum2 = Convert.ToInt32(Console.ReadLine());\n\n\t\tresult = Sum(num1, num2);\n\t\tConsole.WriteLine("Sum: " + result);\n\n\t\tConsole.ReadKey();\n\t}\n\n\tstatic int Sum(int a, int b)\n\t{\n\t\treturn a + b;\n\t}\n}',
    '<s>[INST] Create a leap year checking program in C# [/INST] class LeapYear\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint year;\n\n\t\tConsole.WriteLine("Leap Year Checker\\n");\n\t\tConsole.WriteLine("Enter a year:");\n\t\tyear = Convert.ToInt32(Console.ReadLine());\n\n\t\tif (IsLeapYear(year) == 1)\n\t\t\tConsole.WriteLine(year + " is a leap year.");\n\t\telse\n\t\t\tConsole.WriteLine(year + " is not a leap year.");\n\n\t\tConsole.ReadKey();\n\t}\n\n\tstatic int IsLeapYear(int year)\n\t{\n\t\tint isLeap = 0;\n\n\t\tif ((year % 4 == 0 && year % 100 != 0) || (year % 400 == 0))\n\t\t\tisLeap = 1;\n\n\t\treturn isLeap;\n\t}\n}',
    '<s>[INST] Create an interest calculator in C# [/INST] class InterestCalculator\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tdouble principal, rate, time, interestBasisPoints, interest;\n\n\t\tConsole.WriteLine("Simple Interest Calculator\\n");\n\t\tConsole.WriteLine("Enter the principal amount:\\n");\n\t\tprincipal = Convert.ToDouble(Console.ReadLine());\n\n\t\tConsole.WriteLine("Enter the interest rate (in basis points, e.g., 500 for 5%):\\n");\n\t\trate = Convert.ToDouble(Console.ReadLine());\n\n\t\tConsole.WriteLine("Enter the time (years):\\n");\n\t\ttime = Convert.ToDouble(Console.ReadLine());\n\n\t\tinterestBasisPoints = (principal * rate * time) / 100;\n\t\tinterest = interestBasisPoints / 1000.0;\n\t\tConsole.WriteLine("Interest: " + interest);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create an income tax calculator in C# [/INST] class IncomeTaxCalculator\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tdouble income, tax;\n\n\t\tConsole.WriteLine("Income Tax Calculator\\n");\n\t\tConsole.WriteLine("Enter your income:");\n\t\tincome = Convert.ToDouble(Console.ReadLine());\n\n\t\tif (income <= 10000)\n\t\t\ttax = 0.05 * income;\n\t\telse if (income <= 50000)\n\t\t\ttax = 0.1 * income;\n\t\telse if (income <= 100000)\n\t\t\ttax = 0.2 * income;\n\t\telse\n\t\t\ttax = 0.3 * income;\n\n\t\tConsole.WriteLine("Income: " + income);\n\t\tConsole.WriteLine("Tax: " + tax);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that checks if a number is even or odd [/INST] class EvenOrOdd\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint num;\n\n\t\tConsole.WriteLine("Enter a number:");\n\t\tnum = Convert.ToInt32(Console.ReadLine());\n\n\t\tif (num % 2 == 0)\n\t\t\tConsole.WriteLine(num + " is even.");\n\t\telse\n\t\t\tConsole.WriteLine(num + " is odd.");\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that finds the largest number among three numbers [/INST] class LargestNumber\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint num1, num2, num3, largest;\n\n\t\tConsole.WriteLine("Enter three numbers:");\n\t\tnum1 = Convert.ToInt32(Console.ReadLine());\n\t\tnum2 = Convert.ToInt32(Console.ReadLine());\n\t\tnum3 = Convert.ToInt32(Console.ReadLine());\n\n\t\tlargest = num1 > num2 ? (num1 > num3 ? num1 : num3) : (num2 > num3 ? num2 : num3);\n\t\tConsole.WriteLine("The largest number is: " + largest);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that calculates the factorial of a number [/INST] class Factorial\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint num, fact = 1;\n\n\t\tConsole.WriteLine("Enter a number:");\n\t\tnum = Convert.ToInt32(Console.ReadLine());\n\n\t\tfor (int i = 1; i <= num; i++)\n\t\t\tfact *= i;\n\n\t\tConsole.WriteLine("Factorial of " + num + " is: " + fact);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that converts temperature from Celsius to Fahrenheit [/INST] class TemperatureConverter\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tdouble celsius, fahrenheit;\n\n\t\tConsole.WriteLine("Enter temperature in Celsius:");\n\t\tcelsius = Convert.ToDouble(Console.ReadLine());\n\n\t\tfahrenheit = (celsius * 9 / 5) + 32;\n\t\tConsole.WriteLine("Temperature in Fahrenheit: " + fahrenheit);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that calculates the average of five numbers [/INST] class AverageCalculator\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint num1, num2, num3, num4, num5;\n\t\tdouble average;\n\n\t\tConsole.WriteLine("Enter five numbers:");\n\t\tnum1 = Convert.ToInt32(Console.ReadLine());\n\t\tnum2 = Convert.ToInt32(Console.ReadLine());\n\t\tnum3 = Convert.ToInt32(Console.ReadLine());\n\t\tnum4 = Convert.ToInt32(Console.ReadLine());\n\t\tnum5 = Convert.ToInt32(Console.ReadLine());\n\n\t\taverage = (num1 + num2 + num3 + num4 + num5) / 5.0;\n\t\tConsole.WriteLine("Average: " + average);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that checks if a number is prime or not [/INST] class PrimeCheck\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint num, flag = 0;\n\n\t\tConsole.WriteLine("Enter a number:");\n\t\tnum = Convert.ToInt32(Console.ReadLine());\n\n\t\tfor (int i = 2; i <= num / 2; i++)\n\t\t{\n\t\t\tif (num % i == 0)\n\t\t\t{\n\t\t\t\tflag = 1;\n\t\t\t\tbreak;\n\t\t\t}\n\t\t}\n\n\t\tif (flag == 0)\n\t\t\tConsole.WriteLine(num + " is a prime number.");\n\t\telse\n\t\t\tConsole.WriteLine(num + " is not a prime number.");\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that reverses a given number [/INST] class ReverseNumber\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint num, reversed = 0;\n\n\t\tConsole.WriteLine("Enter a number:");\n\t\tnum = Convert.ToInt32(Console.ReadLine());\n\n\t\twhile (num != 0)\n\t\t{\n\t\t\treversed = reversed * 10 + num % 10;\n\t\t\tnum /= 10;\n\t\t}\n\n\t\tConsole.WriteLine("Reversed number: " + reversed);\n\n\t\tConsole.ReadKey();\n\t}\n}',
    '<s>[INST] Create a program in C# that prints the Fibonacci series up to a given number of terms [/INST] class FibonacciSeries\n{\n\tstatic void Main(string[] args)\n\t{\n\t\tint n, first = 0, second = 1, next;\n\n\t\tConsole.WriteLine("Enter the number of terms:");\n\t\tn = Convert.ToInt32(Console.ReadLine());\n\n\t\tConsole.WriteLine("Fibonacci Series:");\n\t\tfor (int i = 0; i < n; i++)\n\t\t{\n\t\t\tConsole.Write(first + " ");\n\t\t\tnext = first + second;\n\t\t\tfirst = second;\n\t\t\tsecond = next;\n\t\t}\n\n\t\tConsole.ReadKey();\n\t}\n}'
]
dataset = [{"text": tune} for tune in fine_tune]
df = pd.DataFrame(dataset)
mam_dataset = Dataset.from_pandas(df)

In [6]:
# This is for fine tuning

In [1]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      # uses the number of epochs earlier
    per_device_train_batch_size=4,          # 4 seems reasonable
    gradient_accumulation_steps=2,          # 2 is fine, as we're a small batch
    optim="paged_adamw_32bit",              # default optimizer
    save_steps=0,                           # we're not gonna save
    logging_steps=10,                       # same value as used by Meta
    learning_rate=2e-4,                     # standard learning rate
    weight_decay=0.001,                     # standard weight decay 0.001
    fp16=False,                             # set to true for A100
    bf16=False,                             # set to true for A100
    max_grad_norm=0.3,                      # standard setting
    max_steps=-1,                           # needs to be -1, otherwise overrides epochs
    warmup_ratio=0.03,                      # standard warmup ratio
    group_by_length=True,                   # speeds up the training
    lr_scheduler_type="cosine",           # constant seems better than cosine
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=mam_dataset,
    peft_config=peft_config,                # use our lora peft config
    dataset_text_field="text",
    max_seq_length=None,                    # no max sequence length
    tokenizer=tokenizer,                    # use the llama tokenizer
    args=training_arguments,                # use the training arguments
    packing=False,                          # don't need packing
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(fine_tuned_model_name)
trainer.tokenizer.save_pretrained(fine_tuned_model_name)

In [8]:
# This is the fine tuned model

In [9]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    fine_tuned_model_name,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(
    fine_tuned_model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
logging.set_verbosity(logging.CRITICAL)
prompt = "Create a program for insertion sort."
pipe = pipeline(task="text-generation", model=model,
                tokenizer=tokenizer, max_length=500)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

/home/aasif/.local/lib/python3.10/site-packages/transformers/generation/utils.py:1473: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(


<s>[INST] Create a program for insertion sort. [/INST] class InsertionSort
{
	static void Main(string[] args)
	{
		int[] arr = { 5, 1, 4, 2, 3 };

		for (int i = 1; i < arr.Length; i++)
		{
			int key = arr[i];
			int j = i - 1;

			while (j >= 0 && arr[j] > key)
			{
				arr[j + 1] = arr[j];
				j--;
			}

			arr[j + 1] = key;
		}

		foreach (int i in arr)
			Console.WriteLine(i);

		Console.ReadKey();
	}
}

class InsertionSort
{
	static void Main(string[] args)
	{
		int[] arr = { 5, 1, 4, 2, 3 };

		for (int i = 1; i < arr.Length; i++)
		{
			int key = arr[i];
			int j = i - 1;

			while (j >= 0 && arr[j] > key)
			{
				arr[j + 1] = arr[j];
				j--;
			}

			arr[j + 1] = key;
		}

		foreach (int i in arr)
			Console.WriteLine(i);

		Console.ReadKey();
	}
} 

// Insertion Sort
// An insertion sort is a simple sorting algorithm that works similar to how people sort cards in their hands. 
// First, a card is picked, then it is placed at the correct position in the sorted sequence. 
// 